# Prediction of COVID-19 around the world

Student: Angela Amador

TMU Student Number: 500259095

Supervisor: Tamer Abdou, PhD


I aim to demonstrate how Machine Learning (ML) models were able to predict the spread of COVID-19 around the world.

First, I will explore the dataset to get insides and better understand patterns, detect error and outliers, and find relationships between variables. 


## Preparation
Describing the working dataset and any imposed constraints

This dataset is taken from Our World in Data website, officially collected by Our World in Data team: https://covid.ourworldindata.org/data/owid-covid-data.csv.

This dataset will be synced daily. For more info: https://www.kaggle.com/datasets/caesarmario/our-world-in-data-covid19-dataset

In [1]:
# Import libraries
import pandas as pd
from ydata_profiling import ProfileReport
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from sklearn.feature_selection import VarianceThreshold

import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix



### Load file and explore data

The dataset, provided by Our World in Data, provides COVID-19 information collected by Our World in Data available to Kaggle community https://www.kaggle.com/datasets/caesarmario/our-world-in-data-covid19-dataset/download?datasetVersionNumber=418. This dataset is updated daily, for the purpose of this study I am analyzing the data with information up to Oct 7th, 2023.

In [2]:
# Load file
covid_data = pd.read_csv('archive.zip', sep=',')  

#Explore data
covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


### Check the data type and metadata of the attributes

In [3]:
covid_data.dtypes

iso_code                                    object
continent                                   object
location                                    object
date                                        object
total_cases                                float64
                                            ...   
population                                 float64
excess_mortality_cumulative_absolute       float64
excess_mortality_cumulative                float64
excess_mortality                           float64
excess_mortality_cumulative_per_million    float64
Length: 67, dtype: object

In [4]:
# look at meta information about data, such as null values
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346567 entries, 0 to 346566
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    346567 non-null  object 
 1   continent                                   330089 non-null  object 
 2   location                                    346567 non-null  object 
 3   date                                        346567 non-null  object 
 4   total_cases                                 308672 non-null  float64
 5   new_cases                                   337028 non-null  float64
 6   new_cases_smoothed                          335769 non-null  float64
 7   total_deaths                                287169 non-null  float64
 8   new_deaths                                  337072 non-null  float64
 9   new_deaths_smoothed                         335842 non-null  float64
 

In [5]:
# Let's see meta information about numeric data, we can also see if there any extreme values
covid_data.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,3.086720e+05,3.370280e+05,3.357690e+05,2.871690e+05,337072.000000,335842.000000,308672.000000,337028.000000,335769.000000,287169.000000,...,198833.000000,131627.000000,237221.000000,318823.000000,260466.000000,3.465670e+05,1.195300e+04,11953.000000,11953.000000,11953.000000
mean,6.609069e+06,9.695906e+03,9.732069e+03,8.559525e+04,86.392889,86.704207,100634.394008,146.569024,147.113196,867.354640,...,32.909864,50.789455,3.097109,73.714185,0.722460,1.283225e+08,5.113535e+04,9.739424,11.461129,1646.844959
std,4.032547e+07,1.108324e+05,9.495414e+04,4.380493e+05,616.815791,561.926045,150292.226515,1169.506821,602.840371,1096.750172,...,13.574185,31.956355,2.548353,7.395560,0.148979,6.603117e+08,1.442796e+05,12.380781,25.354695,1929.159161
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.700000,1.188000,0.100000,53.280000,0.394000,4.700000e+01,-3.772610e+04,-44.230000,-95.920000,-2752.924800
25%,7.988750e+03,0.000000e+00,2.860000e-01,1.250000e+02,0.000000,0.000000,2573.783500,0.000000,0.056000,59.672000,...,22.600000,20.859000,1.300000,69.590000,0.602000,4.490020e+05,1.066000e+02,1.320000,-1.620000,65.345720
50%,6.904700e+04,2.000000e+00,2.585700e+01,1.313000e+03,0.000000,0.143000,27720.494000,0.169000,6.815000,374.322000,...,33.100000,49.839000,2.500000,75.050000,0.740000,5.882259e+06,5.736601e+03,8.070000,5.770000,1072.472700
75%,7.345502e+05,2.730000e+02,5.107140e+02,1.181800e+04,3.000000,5.286000,131483.602000,36.566000,84.076000,1356.019000,...,41.300000,82.502000,4.200000,79.460000,0.829000,2.830170e+07,3.668959e+04,15.470000,16.520000,2704.933800
max,7.711505e+08,8.401961e+06,6.402036e+06,6.960770e+06,27939.000000,14821.857000,737554.506000,228872.025000,37241.781000,6511.209000,...,78.100000,100.000000,13.800000,86.750000,0.957000,7.975105e+09,1.289776e+06,76.550000,377.630000,10292.916000


### Removing data before COVID vaccinate availability

Multiple vaccinates became available on the second semester of 2020. By December most contries have approved vaccinates for their own country. 

To avoid ..xxxxxx... we will remove data before Jan 1st, 2021 to consider data only with vaccinate availability

In [6]:
print("Original dataset:")
print("Total number of observations: ", covid_data.shape[0])
print("Total number of attributes: ", covid_data.shape[1])
print("Size: ", covid_data.size)


covid_data = covid_data.drop(covid_data[covid_data.date < '2021-01-01'].index)

print("\nAfter removing data before vaccinate was available around the world (Jan 1st, 2021):")
print("Total number of observations: ", covid_data.shape[0])
print("Total number of attributes: ", covid_data.shape[1])
print("Size: ", covid_data.size)


Original dataset:
Total number of observations:  346567
Total number of attributes:  67
Size:  23219989

After removing data before vaccinate was available around the world (Jan 1st, 2021):
Total number of observations:  255173
Total number of attributes:  67
Size:  17096591


## Categorical attributes

Base of the analysis of the attributes iso_code and location, I can tell that one can be derive from the other. For the purpose of this study, I am going to keep location and remove iso_code

In [7]:
# import pickle
# with open("data.pickle", "wb") as output:
#     pickle.dump(covid_data, output, pickle.HIGHEST_PROTOCOL)

# with open("data.pickle", "rb") as input:
#     data = pickle.load(input)

covid_data.groupby(["iso_code"])["iso_code"].count()
covid_data.groupby(["location"])["location"].count()

covid_data = covid_data.drop(['iso_code'], axis=1)

print("\nAfter removing attribute iso_code because it can be delivered from location:")
print("Total number of observations: ", covid_data.shape[0])
print("Total number of attributes: ", covid_data.shape[1])
print("Size: ", covid_data.size)

iso_code
ABW    1009
AFG    1007
AGO    1007
AIA    1007
ALB    1007
       ... 
WSM    1007
YEM    1007
ZAF    1007
ZMB    1007
ZWE    1007
Name: iso_code, Length: 255, dtype: int64

location
Afghanistan       1007
Africa            1007
Albania           1007
Algeria           1007
American Samoa    1007
                  ... 
Western Sahara       1
World             1010
Yemen             1007
Zambia            1007
Zimbabwe          1007
Name: location, Length: 255, dtype: int64


After removing attribute iso_code because it can be delivered from location:
Total number of observations:  255173
Total number of attributes:  66
Size:  16841418


# Dimensional Reduction (CMTH642 - Module 9)

Due to the size of the dataset with 255,173 entries and 67 columns, I am going to apply dimensional reduction to provide better features for statistical learning methods

## 1. Removing data columns with too many NaN values

We can calculate the ratio of missing values using a simple formula. The formula is- the number of missing values in each column divided by the total number of observation. Generally, we can drop variables having a missing value ratio of more than 60% or 70%. For my purpose I am going to use a threashold of 60% missing values and remove those attributes.

In [8]:
# Defining threashold of 60% missing values 
threashold_NaN = 0.60

#Explore data
def describe_nan(df):
    return pd.DataFrame([(i, df[df[i].isna()].shape[0],df[df[i].isna()].shape[0]/df.shape[0]) for i in df.columns], columns=['column', 'nan_counts', 'nan_rate'])

pd.options.display.max_rows = None

#icu=covid_data.icu_patients.value_counts(dropna=False)
#display ("NaN entries for the icu_patients column:", icu[icu.index.isnull()])

print("Attributes with more than 60 percentage of missing values:")

describe_nan(covid_data).sort_values(by="nan_rate", ascending=False).query("nan_rate >= %s"%threashold_NaN)

#((covid_data.isnull() | covid_data.isna()).sum() * 100 / covid_data.index.size).round(2)

Attributes with more than 60 percentage of missing values:


,column,nan_counts,nan_rate
65,excess_mortality_cumulative_per_million,246760,0.967030
64,excess_mortality,246760,0.967030
63,excess_mortality_cumulative,246760,0.967030
62,excess_mortality_cumulative_absolute,246760,0.967030
20,weekly_icu_admissions,246705,0.966815
21,weekly_icu_admissions_per_million,246705,0.966815
23,weekly_hosp_admissions_per_million,237173,0.929460
22,weekly_hosp_admissions,237173,0.929460
17,icu_patients_per_million,226284,0.886787
16,icu_patients,226284,0.886787


In [9]:

my_columns = describe_nan(covid_data).sort_values(by="nan_rate", ascending=False).query("nan_rate < %s"%threashold_NaN)[["column"]]
my_columns = my_columns['column'].to_list() 

#dr1 -> Dimensionality Reduction - 1. Removing data columns with too many missing values
dr1_covid_data = covid_data[my_columns]

print("After removing columns with more than 60 percentage of missing values:\n")
print("Total number of observations: ", dr1_covid_data.shape[0])
print("Total number of attributes: ", dr1_covid_data.shape[1])
print("Size: ", dr1_covid_data.size)
print("\n")
dr1_covid_data.info()

After removing columns with more than 60 percentage of missing values:

Total number of observations:  255173
Total number of attributes:  35
Size:  8931055


<class 'pandas.core.frame.DataFrame'>
Index: 255173 entries, 364 to 346566
Data columns (total 35 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   extreme_poverty                             126910 non-null  float64
 1   stringency_index                            132092 non-null  float64
 2   reproduction_rate                           137489 non-null  float64
 3   male_smokers                                146049 non-null  float64
 4   female_smokers                              148063 non-null  float64
 5   hospital_beds_per_thousand                  174245 non-null  float64
 6   new_people_vaccinated_smoothed              179462 non-null  float64
 7   new_people_vaccinated_smoothed_per_hundred  179462 non-null  f

In [10]:
print("Percentage of NaN values per attribute for the remaining columns:\n")
describe_nan(dr1_covid_data).sort_values(by="nan_rate", ascending=False)

# To manage memory dur to the size of the dataset, I am keeping one version of the dataset and removing any temporary copy
covid_data = dr1_covid_data
del(dr1_covid_data)

Percentage of NaN values per attribute for the remaining columns:



,column,nan_counts,nan_rate
0,extreme_poverty,128263,0.502651
1,stringency_index,123081,0.482343
2,reproduction_rate,117684,0.461193
3,male_smokers,109124,0.427647
4,female_smokers,107110,0.419754
5,hospital_beds_per_thousand,80928,0.317150
6,new_people_vaccinated_smoothed,75711,0.296705
7,new_people_vaccinated_smoothed_per_hundred,75711,0.296705
9,new_vaccinations_smoothed_per_million,75551,0.296078
8,new_vaccinations_smoothed,75551,0.296078


## 2. Low Variance Filter

Another way of measuring how much information a data column has, is to measure its variance. In the limit case where the column cells assume a constant value, the variance would be 0 and the column would be of no help in the discrimination of different groups of data.

The Low Variance Filter node calculates each column variance and removes those columns with a variance value below a given threshold. Notice that the variance can only be calculated for numerical columns, i.e. this dimensionality reduction method applies only to numerical columns. Note, too, that the variance value depends on the column numerical range. Therefore data column ranges need to be normalized to make variance values independent from the column domain range.

First a Normalizer node normalizes all column ranges to [0, 1]; next, a Low Variance Filter node calculates the columns variance and filters out the columns with a variance lower than a set threshold.

In [11]:
# Initialization is just like any other Scikit-learn estimator. The default value for the threshold is always 0. 
# Also, the estimator only works with numeric data obviously and it will raise an error if there are categorical features present in the dataframe. 
# That’s why, for now, I will subset the numeric features into another dataframe:

vt = VarianceThreshold()

#dr2 -> Dimensionality Reduction - 2. Removing low variance filter
dr2_covid_data_num = covid_data.select_dtypes(include="number")
#dr2_covid_data_num.shape
#dr2_covid_data_num.info()


In [12]:
# Before, I need to take care of missing values encoded as NaN natively by replacing with the mean on reduced dataset "dr2_covid_data_reduced"

print ("Before replacing NaN values with the mean:\n")
print("Total number of observations: ", dr2_covid_data_num.shape[0])
print("Total number of attributes: ", dr2_covid_data_num.shape[1])
print("Size: ", dr2_covid_data_num.size)
print("\n")
dr2_covid_data_num.info()

for c in dr2_covid_data_num.columns:
    dr2_covid_data_num[c] = dr2_covid_data_num[c].fillna(dr2_covid_data_num[c].mean())

print ("\nAfter replacing NaN values with the mean:\n")
print("Total number of observations: ", dr2_covid_data_num.shape[0])
print("Total number of attributes: ", dr2_covid_data_num.shape[1])
print("Size: ", dr2_covid_data_num.size)
print("\n")
dr2_covid_data_num.info()

Before replacing NaN values with the mean:

Total number of observations:  255173
Total number of attributes:  32
Size:  8165536


<class 'pandas.core.frame.DataFrame'>
Index: 255173 entries, 364 to 346566
Data columns (total 32 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   extreme_poverty                             126910 non-null  float64
 1   stringency_index                            132092 non-null  float64
 2   reproduction_rate                           137489 non-null  float64
 3   male_smokers                                146049 non-null  float64
 4   female_smokers                              148063 non-null  float64
 5   hospital_beds_per_thousand                  174245 non-null  float64
 6   new_people_vaccinated_smoothed              179462 non-null  float64
 7   new_people_vaccinated_smoothed_per_hundred  179462 non-null  float64
 8   new_vaccinations

In [13]:
# First, we fit the estimator to data and call its get_support() method. It returns a boolean mask with True values for columns which are not dropped. 
# We can then use this mask to subset our DataFrame like so

_ = vt.fit(dr2_covid_data_num)
mask = vt.get_support()

dr2_covid_data_num = dr2_covid_data_num.loc[:, mask]

# dr2_covid_data_num.shape

# dr2_covid_data_num.info()


In [14]:
# We still have the same number of features. Now, let’s drop features with variances close to 0
vt = VarianceThreshold(threshold=1)

# Fit
_ = vt.fit(dr2_covid_data_num)

# # Get the boolean mask
mask = vt.get_support()

dr2_covid_data_reduced = dr2_covid_data_num.loc[:, mask]

print ("\nAfter dropping features with variances close to 0:\n")
print("Total number of observations: ", dr2_covid_data_reduced.shape[0])
print("Total number of attributes: ", dr2_covid_data_reduced.shape[1])
print("Size: ", dr2_covid_data_reduced.size)
print("\n")
dr2_covid_data_reduced.info()

# With a threshold of 1, 3 attributes were removedthreshold
# From: (255173, 32)
# To: (255173, 29)


After dropping features with variances close to 0:

Total number of observations:  255173
Total number of attributes:  29
Size:  7400017


<class 'pandas.core.frame.DataFrame'>
Index: 255173 entries, 364 to 346566
Data columns (total 29 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   extreme_poverty                        255173 non-null  float64
 1   stringency_index                       255173 non-null  float64
 2   male_smokers                           255173 non-null  float64
 3   female_smokers                         255173 non-null  float64
 4   hospital_beds_per_thousand             255173 non-null  float64
 5   new_people_vaccinated_smoothed         255173 non-null  float64
 6   new_vaccinations_smoothed              255173 non-null  float64
 7   new_vaccinations_smoothed_per_million  255173 non-null  float64
 8   aged_65_older                          255173 non-null  f

In [15]:
# The attributes that were dropped are:
# - reproduction_rate
# - new_people_vaccinated_smoothed_per_hundred
# - human_development_index

In [16]:
# Method of normalizing all features by dividing them by their mean

normalized_df = dr2_covid_data_num / dr2_covid_data_num.mean()
normalized_df.head()

print("Variance of the normalized dataset:\n")
normalized_df.var()

,extreme_poverty,stringency_index,reproduction_rate,male_smokers,female_smokers,hospital_beds_per_thousand,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,new_vaccinations_smoothed,new_vaccinations_smoothed_per_million,...,total_cases_per_million,new_cases_per_million,new_cases,new_cases_smoothed,new_cases_smoothed_per_million,new_deaths_smoothed_per_million,new_deaths_smoothed,new_deaths,new_deaths_per_million,population
364,1.0,0.309718,0.991848,1.0,1.0,0.161439,1.0,1.0,1.0,1.0,...,0.009977,0.024114,0.015494,0.011077,0.017253,0.233626,0.110464,0.141538,0.299663,0.321068
365,1.0,0.309718,0.980315,1.0,1.0,0.161439,1.0,1.0,1.0,1.0,...,0.009990,0.009621,0.006181,0.009919,0.015446,0.223424,0.105438,0.117949,0.249377,0.321068
366,1.0,0.309718,0.980315,1.0,1.0,0.161439,1.0,1.0,1.0,1.0,...,0.010014,0.016211,0.010414,0.010389,0.016182,0.223424,0.105438,0.117949,0.249377,0.321068
367,1.0,0.309718,0.980315,1.0,1.0,0.161439,1.0,1.0,1.0,1.0,...,0.010052,0.026358,0.016933,0.010884,0.016950,0.212202,0.100412,0.106154,0.224747,0.321068
368,1.0,0.309718,0.968782,1.0,1.0,0.161439,1.0,1.0,1.0,1.0,...,0.010071,0.013442,0.008636,0.010426,0.016236,0.194859,0.092047,0.082564,0.174461,0.321068


Variance of the normalized dataset:



extreme_poverty                                 1.047002
stringency_index                                0.163080
reproduction_rate                               0.114146
male_smokers                                    0.097370
female_smokers                                  0.579030
hospital_beds_per_thousand                      0.462300
new_people_vaccinated_smoothed                 36.227829
new_people_vaccinated_smoothed_per_hundred      3.587963
new_vaccinations_smoothed                      29.976837
new_vaccinations_smoothed_per_million           1.823817
human_development_index                         0.031885
aged_65_older                                   0.372831
gdp_per_capita                                  0.860112
cardiovasc_death_rate                           0.161974
aged_70_older                                   0.440998
median_age                                      0.069842
diabetes_prevalence                             0.270828
population_density             

In [17]:
# Now, we can use the estimator with a lower threshold like 0.005
vt = VarianceThreshold(threshold=0.005)

# Fit
_ = vt.fit(normalized_df)

# # Get the boolean mask
mask = vt.get_support()

dr2_covid_data_final = dr2_covid_data_num.loc[:, mask]

dr2_covid_data_final.shape

# With a threshold of 0.05, zero attributes were removed threshold
# From: (255173, 32)
# To: (255173, 32)

(255173, 32)

In [18]:
# dr2_covid_data_reduced.columns.get_loc('total_cases')
# dr2_covid_data_reduced.shape[1]

In [19]:
# With method of normalizing no attributes were removed; while with variances close to 0, 3 features were removed.
# - reproduction_rate
# - new_people_vaccinated_smoothed_per_hundred
# - human_development_index

# I will check if it is rigth to removed these 3 attributes. I will test this by training two RandomForestRegressor to predict a total_cases: the first one on the reduced dataset (dr2_covid_data_reduced), feature selected dataset
# and the second one on the full, numeric-feature only dataset (dr2_covid_data_num).

#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import train_test_split

# Find out the index of total_cases column
c = dr2_covid_data_reduced.columns.get_loc('total_cases')

# Find out number of columns
d = dr2_covid_data_reduced.shape[1]

# Build feature, target arrays
X, y = dr2_covid_data_reduced.iloc[:, [i for i in range(d) if i != c]], dr2_covid_data_reduced.iloc[:, [c]]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1121218)

# Init, fit, score
forest = RandomForestRegressor(random_state=1121218)

_ = forest.fit(X_train, y_train)

# Training Score
print(f"Training Score: {forest.score(X_train, y_train)}")
#Training Score: 0.988528867222243

print(f"Test Score: {forest.score(X_test, y_test)}")
# Test Score: 0.9511616691995844

print("Both training and test score suggest a really high performance without overfitting.")

Training Score: 0.9999950801210624
Test Score: 0.9999399219055796
Both training and test score suggest a really high performance without overfitting.


In [20]:
# dr2_covid_data_num.columns.get_loc('total_cases')
# dr2_covid_data_num.shape[1]

In [21]:
# Now, let’s train the same model on the full numeric-only dataset

# Find out the index of total_cases column
c = dr2_covid_data_num.columns.get_loc('total_cases')

# Find out number of columns
d = dr2_covid_data_num.shape[1]


# Build feature, target arrays
X, y = dr2_covid_data_num.iloc[:, [i for i in range(d) if i != c]], dr2_covid_data_num.iloc[:, [c]]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1121218)

# Init, fit, score
forest = RandomForestRegressor(random_state=1121218)

_ = forest.fit(X_train, y_train)

# Training Score
print(f"Training Score: {forest.score(X_train, y_train)}")
#Training Score: 0.988528867222243

print(f"Test Score: {forest.score(X_test, y_test)}")

print("I can confirm that there isn't any impact on the prediction by removing these 3 features")

#Freeing memory
del(X)
del(y)
del(X_train)
del(X_test)
del(y_train)
del(y_test)
del(dr2_covid_data_num)
del(dr2_covid_data_reduced)
del(dr2_covid_data_final)
del(normalized_df)

Training Score: 0.9999920034215055
Test Score: 0.9999035126774154
I can confirm that there isn't any impact on the prediction by removing these 3 features


In [22]:
# Droping the columns identified with variance close to 0
# - reproduction_rate
# - new_people_vaccinated_smoothed_per_hundred
# - human_development_index

covid_data = covid_data.drop(['reproduction_rate'], axis=1)
covid_data = covid_data.drop(['new_people_vaccinated_smoothed_per_hundred'], axis=1)
covid_data = covid_data.drop(['human_development_index'], axis=1)
print("After removing columns identified with variance close to 0:\n")
print("Total number of observations: ", covid_data.shape[0])
print("Total number of attributes: ", covid_data.shape[1])
print("Size: ", covid_data.size)
print("\n")
covid_data.info()

After removing columns identified with variance close to 0:

Total number of observations:  255173
Total number of attributes:  32
Size:  8165536


<class 'pandas.core.frame.DataFrame'>
Index: 255173 entries, 364 to 346566
Data columns (total 32 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   extreme_poverty                        126910 non-null  float64
 1   stringency_index                       132092 non-null  float64
 2   male_smokers                           146049 non-null  float64
 3   female_smokers                         148063 non-null  float64
 4   hospital_beds_per_thousand             174245 non-null  float64
 5   new_people_vaccinated_smoothed         179462 non-null  float64
 6   new_vaccinations_smoothed              179622 non-null  float64
 7   new_vaccinations_smoothed_per_million  179622 non-null  float64
 8   aged_65_older                          194141 non

## 3. High correlation with other data columns


* https://www.kaggle.com/code/bbloggsbott/feature-selection-correlation-and-p-value

In [23]:
# 

#import seaborn as sns
#import matplotlib.pyplot as plt
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#import warnings
#warnings.filterwarnings("ignore")
#from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
#from sklearn.metrics import confusion_matrix

# corr = dr2_covid_data_final.corr()
# corr.head()

# sns.heatmap(corr)

In [24]:
# columns = np.full((corr.shape[0],), True, dtype=bool)
# for i in range(corr.shape[0]):
#     for j in range(i+1, corr.shape[0]):
#         if corr.iloc[i,j] >= 0.9:
#             if columns[j]:
#                 columns[j] = False

# selected_columns = dr2_covid_data_final.columns[columns]
# selected_columns
# selected_columns.shape

* https://towardsdatascience.com/statistics-in-python-collinearity-and-multicollinearity-4cc4dcd82b3f

# Generate Profiling Report

In [25]:
# Genetate profiling report
#profile = ProfileReport(covid_data, title="Profiling Report")
#profile = ProfileReport(covid_data, title="Profiling Report", html={'style':{'fullwith':True}})
#profile